In [7]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import torch
from ultra_data import UltraDataset

# 데이터 경로 설정
data_path = "../datasets/AmazonBooks_m1/"

# 학습 데이터셋 테스트
print("Testing Train Dataset")
train_dataset = UltraDataset(data_path, train=True, validation_split=0)

# 데이터셋 크기 확인
print(f"Train dataset size: {len(train_dataset)}")

# 데이터셋에서 첫 번째 샘플 확인
print("First sample in train dataset:", train_dataset[0])

# 사용자 수와 아이템 수 확인
n_user, m_item = train_dataset.get_user_item_counts()
print(f"Number of users: {n_user}, Number of items: {m_item}")

# 학습 데이터의 희소 행렬 확인
train_matrix = train_dataset.get_train_matrix()
print("Train matrix shape:", train_matrix.shape)

# 제약 행렬 확인
constraint_matrix = train_dataset.get_constraint_matrix()
print("Constraint matrix (beta_uD, beta_iD):", constraint_matrix)

# 테스트 데이터셋 테스트
print("\nTesting Test Dataset")
test_dataset = UltraDataset(data_path, train=False)

# 테스트 데이터셋 크기 확인
print(f"Test dataset size: {len(test_dataset)}")

# 테스트 데이터셋에서 첫 번째 샘플 확인
print("First sample in test dataset:", test_dataset[0])

# 사용자 수와 아이템 수 확인
n_user_test, m_item_test = test_dataset.get_user_item_counts()
print(f"Number of users in test dataset: {n_user_test}, Number of items in test dataset: {m_item_test}")


Testing Train Dataset
Train dataset size: 2380730
First sample in train dataset: (6716, 51773)
Number of users: 52643, Number of items: 91599
Train matrix shape: (52643, 91599)
Constraint matrix (beta_uD, beta_iD): {'beta_uD': tensor([0.1387, 0.0553, 0.1474,  ..., 0.2577, 0.2577, 0.2083]), 'beta_iD': tensor([0.1313, 0.0655, 0.1336,  ..., 0.7071, 0.7071, 0.7071])}

Testing Test Dataset
Test dataset size: 603378
First sample in test dataset: (0, 7202)
Number of users in test dataset: 52643, Number of items in test dataset: 91599


In [3]:
!pwd

/Users/sunukkim/workspace/Research/UltraGCN/src


In [13]:
# Sample test script for UltraDataset class
import os
import torch
from ultra_data import UltraDataset

# Define a sample path where the data files would be located
# Assume 'train.txt' and 'test.txt' files are in this directory
data_path = "../datasets/AmazonBooks_m1/"

# Initialize the dataset with training data
train_dataset = UltraDataset(data_path=data_path, train=True, validation_split=0.1)

# Check the split sizes
print("Total data size:", len(train_dataset.data))
print("Training data size (90%):", len(train_dataset.train_data))
print("Validation data size (10%):", len(train_dataset.get_validation_data()))

# Check if the constraint matrix and train matrix are created
train_matrix = train_dataset.get_train_matrix()
constraint_matrix = train_dataset.get_constraint_matrix()

print("Train Matrix (Sparse):", train_matrix.shape)
print("Constraint Matrix (Beta_uD):", constraint_matrix["beta_uD"].shape)
print("Constraint Matrix (Beta_iD):", constraint_matrix["beta_iD"].shape)

# Validate __getitem__ and __len__ for training data
print("Sample training data point:", train_dataset[0])
print("Total training data points:", len(train_dataset))

# Initialize the dataset with test data
test_dataset = UltraDataset(data_path=data_path, train=False)
print("Test data size:", len(test_dataset))
print("Sample test data point:", test_dataset[0])


Total data size: 2380730
Training data size (90%): 2142657
Validation data size (10%): 238073
Train Matrix (Sparse): (52643, 91599)
Constraint Matrix (Beta_uD): torch.Size([52643])
Constraint Matrix (Beta_iD): torch.Size([91599])
Sample training data point: (15033, 66377)
Total training data points: 2142657
Test data size: 603378
Sample test data point: (0, 7202)


In [2]:
# UltraDataset 테스트 코드
import os
import torch
import numpy as np
from ultra_data import UltraDataset

# 데이터 파일 경로 설정
data_path = "../datasets/AmazonBooks_m1/"

# UltraDataset 인스턴스 생성 (훈련 모드로 설정)
dataset = UltraDataset(data_path=data_path, train=True)

# get_interacted_items 메서드를 통해 상호작용한 아이템과 마스크 가져오기
interacted_items, mask = dataset.get_interacted_items()

# 몇 명의 샘플 사용자에 대한 상호작용한 아이템과 마스크 확인
sample_users = [0, 1, 2]  # 데이터셋의 크기에 맞게 사용자 ID를 조정

print("샘플 사용자의 상호작용 아이템과 마스크:")
for user in sample_users:
    print(f"\n사용자 {user}의 상호작용 아이템:", interacted_items[user])
    print(f"사용자 {user}의 마스크 (마스킹된 인덱스):", torch.nonzero(mask[user] == -np.inf, as_tuple=True)[0].tolist())

# 추가 검증
# interacted_items에 있는 각 아이템이 mask에서 -inf로 마스킹되어 있는지 확인
for user, items in enumerate(interacted_items):
    for item in items:
        assert mask[user][item] == -np.inf, f"오류: 사용자 {user}, 아이템 {item}에 대한 마스크가 -inf가 아닙니다."
print("\n모든 상호작용 아이템이 -inf로 올바르게 마스킹되었습니다.")


샘플 사용자의 상호작용 아이템과 마스크:

사용자 0의 상호작용 아이템: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]
사용자 0의 마스크 (마스킹된 인덱스): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]

사용자 1의 상호작용 아이템: [53, 54, 49, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 26, 75, 76, 77, 78, 79, 80, 81, 82, 83, 14, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 29, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 52, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 22, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,

In [6]:
# 각 사용자의 상호작용 아이템 개수 출력
interacted_counts = [len(items) for items in interacted_items]
print("사용자별 상호작용 아이템 개수:", interacted_counts)
print("총 사용자 수:", len(interacted_counts))

# 전체 상호작용 개수 계산
total_interactions = sum(len(items) for items in interacted_items)
print("전체 상호작용 개수:", total_interactions)

사용자별 상호작용 아이템 개수: [53, 328, 47, 151, 45, 84, 37, 19, 44, 19, 183, 23, 20, 24, 53, 310, 54, 18, 47, 60, 46, 87, 201, 93, 2785, 304, 21, 110, 468, 97, 178, 141, 56, 16, 28, 32, 346, 488, 32, 140, 36, 82, 46, 52, 318, 20, 27, 33, 125, 21, 248, 66, 65, 38, 268, 40, 149, 23, 88, 32, 19, 69, 23, 113, 56, 43, 24, 151, 174, 41, 224, 20, 20, 24, 44, 184, 57, 18, 24, 203, 162, 21, 194, 44, 63, 257, 66, 108, 219, 301, 121, 78, 669, 212, 48, 373, 417, 29, 68, 263, 56, 22, 17, 54, 24, 17, 36, 248, 44, 116, 25, 45, 16, 455, 338, 132, 257, 31, 20, 172, 85, 35, 21, 41, 71, 95, 23, 83, 120, 240, 111, 35, 21, 46, 21, 156, 68, 48, 140, 16, 27, 117, 572, 262, 10682, 19, 20, 296, 62, 182, 18, 115, 16, 61, 553, 20, 23, 26, 267, 184, 705, 888, 31, 203, 184, 16, 80, 16, 25, 1476, 207, 28, 67, 25, 80, 18, 30, 173, 20, 148, 21, 108, 25, 108, 68, 187, 36, 66, 83, 118, 36, 80, 33, 24, 292, 22, 796, 134, 28, 38, 22, 58, 64, 20, 64, 179, 446, 72, 108, 21, 56, 35, 23, 595, 165, 36, 45, 72, 48, 140, 54, 80, 58, 192, 

In [1]:
from ultra_data import UltraDataset

# Test code to check total interaction counts for train and test data
import os

# 데이터 경로 설정 (예: "data")
data_path = "../datasets/AmazonBooks_m1/"  # 데이터 경로에 맞게 수정하세요

# Train 데이터와 Test 데이터를 로드하여 상호작용 총 개수 계산
train_dataset = UltraDataset(data_path, train=True)
test_dataset = UltraDataset(data_path, train=False)

# Train 데이터의 상호작용 개수 계산
train_interaction_count = len(train_dataset.data)
print(f"Total interactions in train data: {train_interaction_count}")

# Test 데이터의 상호작용 개수 계산
test_interaction_count = len(test_dataset.data)
print(f"Total interactions in test data: {test_interaction_count}")

# Optional: Train과 Test 데이터에서 상호작용된 항목들 출력 (확인용)
train_interacted_items, _ = train_dataset.get_interacted_items()
test_interacted_items, _ = test_dataset.get_interacted_items()

# Train 데이터에서 상호작용된 항목 개수 확인 (사용자별로)
train_interacted_counts = [len(items) for items in train_interacted_items]
print(f"Number of interactions per user in train data: {train_interacted_counts[:10]}")  # 처음 10명의 사용자

# Test 데이터에서 상호작용된 항목 개수 확인 (사용자별로)
test_interacted_counts = [len(items) for items in test_interacted_items]
print(f"Number of interactions per user in test data: {test_interacted_counts[:10]}")  # 처음 10명의 사용자


/Users/sunukkim/miniconda3/envs/UltraGCN/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total interactions in train data: 2380730
Total interactions in test data: 603378
Number of interactions per user in train data: [53, 328, 47, 151, 45, 84, 37, 19, 44, 19]
Number of interactions per user in test data: [14, 83, 12, 38, 12, 21, 10, 5, 11, 4]


In [2]:
print(f"Total interactions in data: {train_interaction_count
                                     + test_interaction_count}")

Total interactions in data: 2984108


In [11]:
# 사용자와 아이템 수
n_user, m_item = train_dataset.get_user_item_counts()  # Train과 Test의 사용자, 아이템 수는 동일하다고 가정

# 전체 가능한 상호작용 개수
total_possible_interactions = n_user * m_item

# 실제 상호작용 개수 (train과 test data에서 추출)
actual_interactions = len(train_dataset.data) + len(test_dataset.data)

density_percentage = (actual_interactions / total_possible_interactions) * 100
print(f"Total Dataset Density (Train + Test): {density_percentage:.3f}%")

Total Dataset Density (Train + Test): 0.062%
